In [1]:
import numpy as np
import csv

class test_reader(object):
  def __init__(self, data_file="./data/test.csv"):
    self.value = []
    with open(data_file, "rb") as f:
      csv_reader = csv.reader(f, delimiter=",")
      
      for i, row in enumerate(csv_reader):
        if i == 0:
          self.attribute = row
          continue

        self.value.append(row)
    
    self.raw_to_vector(self.value)
   
  def raw_to_vector(self, value):
    self.x = []
    self.id = []
    for i, row in enumerate(self.value):
      x = np.zeros(6)

      x[0] = float(row[2-1]) # Pclass
      x[1] = 0 if row[4-1] == "male" else 1 # Sex
      try: x[2] = float(row[5-1]) # Age
      except: x[2] = 20.0
      x[3] = float(row[6-1]) # SibSp
      x[4] = float(row[7-1]) # Parch
      try: x[5] = float(row[9-1]) # Fare
      except: x[5] = 0.0 # If not fare exists, initialize it to zero

      id = int(row[0]) # Passenger ID

      self.x.append(x)
      self.id.append(id)

    self.x, self.id = np.array(self.x), np.array(self.id)

  def full_batch(self):
    return self.x, self.id


class reader(object):
  def __init__(self, data_file = "./data/train.csv"):
    self.value = []
    with open(data_file, "rb") as f:
      csv_reader = csv.reader(f, delimiter=",")
      
      for i, row in enumerate(csv_reader):
        if i == 0:
          self.attribute = row
          continue

        self.value.append(row)
    
    self.raw_to_vector(self.value)
    self.split(num_validation_examples=100)

    # self.x_train.shape = (791, 6)
    # self.y_train.shape = (791)
    # self.id_train.shape = (791)
    
    # self.x_val.shape = (100, 6)
    # self.y_val.shape = (100)
    # self.id_val.shape = (100)

    self.num_examples = len(self.x_train) # = 791 (train_data)
    self.start_index = 0
    self.shuffle_indices = range(self.num_examples)

    self.num_examples_val = len(self.x_val) # = 100 (validation_data)
    self.start_index_val = 0
    self.shuffle_indices_val = range(self.num_examples_val)

  def raw_to_vector(self, value):
    self.x = []
    self.y = []
    self.id = []
    for i, row in enumerate(self.value):
      x = np.zeros(6)

      x[0] = float(row[2]) # Pclass
      x[1] = 0 if row[4] == "male" else 1 # Sex
      try: x[2] = float(row[5]) # Age
      except: x[2] = 20.0
      x[3] = float(row[6]) # SibSp
      x[4] = float(row[7]) # Parch
      x[5] = float(row[9]) # Fare

      y = int(row[1]) # Survived = (0, 1)
      id = int(row[0]) # Passenger ID

#       if i < 10:
#         print "x: ", x, "/shape: ", np.array(x).shape
#         print "y: ", y, "/shape: ", np.array(y).shape
#         print "id: ", id, "/shape: ", np.array(id).shape

      self.x.append(x)
      self.y.append(y)
      self.id.append(id)

    self.x, self.y, self.id = np.array(self.x), np.array(self.y), np.array(self.id)

  def split(self, num_validation_examples):
    self.x_train = self.x[ num_validation_examples: ]
    self.x_val = self.x[ : num_validation_examples ]

    self.y_train = self.y[ num_validation_examples: ]
    self.y_val = self.y[ : num_validation_examples ]

    self.id_train = self.id[ num_validation_examples: ]
    self.id_val = self.id[ :num_validation_examples ]

  def next_batch(self, batch_size, split="train"):

    if split == "train":
      if self.start_index == 0:
        np.random.shuffle(self.shuffle_indices) # shuffle indices

      end_index = min([self.num_examples, self.start_index + batch_size])
      batch_indices = [ self.shuffle_indices[idx] for idx in range(self.start_index, end_index) ]

      batch_x = self.x_train[ batch_indices ]
      batch_y = self.y_train[ batch_indices ]
      batch_id = self.id_train[ batch_indices ] 

      if end_index == self.num_examples:
        self.start_index = 0
      else: self.start_index = end_index

      return batch_x, batch_y, batch_id 

    elif split == "val":
      if self.start_index_val == 0:
        np.random.shuffle(self.shuffle_indices_val) # shuffle indices

      end_index = min([self.num_examples_val, self.start_index_val + batch_size])
      batch_indices = [ self.shuffle_indices_val[idx] for idx in range(self.start_index_val, end_index) ]

      batch_x = self.x_val[ batch_indices ]
      batch_y = self.y_val[ batch_indices ]
      batch_id = self.id_val[ batch_indices ] 

      if end_index == self.num_examples_val:
        self.start_index_val = 0
      else: self.start_index_val = end_index

      return batch_x, batch_y, batch_id  


In [2]:
a = reader()
a.next_batch(32, split="train")

(array([[   2.    ,    0.    ,   18.    ,    0.    ,    0.    ,   11.5   ],
        [   2.    ,    1.    ,   57.    ,    0.    ,    0.    ,   10.5   ],
        [   3.    ,    0.    ,    6.    ,    0.    ,    1.    ,   12.475 ],
        [   3.    ,    0.    ,   24.    ,    0.    ,    0.    ,    7.05  ],
        [   3.    ,    0.    ,   20.    ,    8.    ,    2.    ,   69.55  ],
        [   2.    ,    1.    ,   50.    ,    0.    ,    0.    ,   10.5   ],
        [   1.    ,    0.    ,   34.    ,    0.    ,    0.    ,   26.55  ],
        [   3.    ,    0.    ,   34.    ,    0.    ,    0.    ,    8.05  ],
        [   1.    ,    0.    ,   19.    ,    1.    ,    0.    ,   53.1   ],
        [   1.    ,    1.    ,   35.    ,    1.    ,    0.    ,   90.    ],
        [   1.    ,    1.    ,   44.    ,    0.    ,    0.    ,   27.7208],
        [   2.    ,    1.    ,   20.    ,    0.    ,    0.    ,   12.35  ],
        [   2.    ,    0.    ,   20.    ,    0.    ,    0.    ,    0.    ],
        [   

In [3]:
import tensorflow as tf

class dnn(object):
  def __init__(self):
    # Define model's input as tf.placeholder
    self.x = tf.placeholder(dtype=tf.float32, shape=[None, 6])
    self.y = tf.placeholder(dtype=tf.int32, shape=[None])

    
    # Define parameters for 3-layers dnn model
    self.w_1, self.b_1 = tf.get_variable(name="w_1", shape=[6, 50]), tf.get_variable(name="b_1", shape=[50])
    self.w_2, self.b_2 = tf.get_variable(name="w_2", shape=[50, 50]), tf.get_variable(name="b_2", shape=[50])
    self.w_3, self.b_3 = tf.get_variable(name="w_3", shape=[50, 2]), tf.get_variable(name="b_3", shape=[2])


    # build graph for forward & backward propagation
    # Also, build_graph() returns scalar and histogram
    self.scalar_summaries, self.histogram_summaries = self.build_graph()

  def build_graph(self):

    # h1.shape = (batch, 50)
    h1 = tf.matmul(self.x, self.w_1) + self.b_1 
    h1 = tf.nn.relu(h1)

    # h2.shape = (batch, 50)
    h2 = tf.matmul(h1, self.w_2) + self.b_2
    h2 = tf.nn.relu(h2)

    # h3.shape = (batch, 2)
    h3 = tf.matmul(h2, self.w_3) + self.b_3
    h3 = tf.nn.softmax(h3, dim=-1)

    prediction = h3

    y_onehot = tf.one_hot(
      indices=self.y,
      depth=2,
      on_value=1.0,
      off_value=0.0,
      )

    # We use L2 loss as cost function and average the batch's loss
    self.loss = tf.reduce_mean((y_onehot - prediction) * (y_onehot - prediction))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    
    self.train_op = optimizer.minimize(self.loss)

    pred_index = tf.cast(tf.argmax(prediction, dimension=1), tf.int32)
    correct_prediction = tf.equal(pred_index, self.y)
    self.acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


    # Save tensors that would be shown on TensorBoard as list 
    scalar_summaries = [(self.loss, "loss"), (self.acc, "accuracy")]
    histogram_summaries = [(h1, "h1"), (h2, "h2"), (h3, "h3"), (self.w_1, "w_1"), (self.b_1, "b_1"), (self.w_2, "w_2"), (self.b_2, "b_2"), (self.w_3, "w_3"), (self.b_3, "b_3")]

    return scalar_summaries, histogram_summaries



In [4]:
import tensorflow as tf

def summary_register(scalar_summaries, histogram_summaries):
  """
  Args:
    scalar_summaries: List of tensors that would be shown to tensorboard as scalar graph
    histogram_summaries: List of tensors that would be shown to tensorboard as histogram    graph

  Returns:
    It only register summary information on TensorFlow Graph, returns nothing.
  """
  for scalar_tensor, name in scalar_summaries:
    tf.summary.scalar(name, scalar_tensor)

  for histogram_tensor, name in histogram_summaries:
    tf.summary.histogram(name, histogram_tensor)



In [5]:
import tensorflow as tf

# Loading data reader at Practice(2)
data_reader = reader()

# Loading DNN graph at Practice(3)
model = dnn()

# Define Session for running graph
# and initialize model's parameters
sess = tf.Session()

# Define op merging all summaries
summary_register(model.scalar_summaries, model.histogram_summaries)
merge_op = tf.summary.merge_all()

# Define Summary Writer for both training and validation phase
train_summary_writer= tf.summary.FileWriter("./logs/train")
val_summary_writer= tf.summary.FileWriter("./logs/val")

sess.run(tf.global_variables_initializer())

batch_size = 16
max_steps = 10000


for i in range(max_steps):
  # For each iteration, first we get batch x&y data
  x_train, y_train, id_train = data_reader.next_batch(16, split="train")

  # Next, construct feed for model's placeholder
  # feed is dictionary whose key is placeholder, and value is feeded value(numpy array)
  feed = {model.x: x_train, model.y: y_train}

  # Go training via running train_op with feeded data!
  # We run simultaneously train_op(backprop) and loss value
  _, loss = sess.run([model.train_op, model.loss], feed_dict=feed)

  if i%10 == 0:
    # Writing training phase summary
    summary = sess.run(merge_op, feed_dict=feed)
    train_summary_writer.add_summary(summary, i)


  # print loss stat every 100 iterations
  if i%100 == 0:
    print "| steps %07d | loss: %.3lf" % (i, loss)


#   running validation process every 100 iterations
  if i%100 == 0:
    x_val, y_val, id_val = data_reader.next_batch(100, split="val")
    feed_val = {model.x: x_val, model.y: y_val}

    validation_acc, summary = sess.run([model.acc, merge_op], feed_dict=feed_val)

    # Writing validation phase summary
    val_summary_writer.add_summary(summary, i)
    print "| steps %07d | Validation Accuracy: %.3lf" % (i, validation_acc)


| steps 0000000 | loss: 0.260
| steps 0000000 | Validation Accuracy: 0.500
| steps 0000100 | loss: 0.249
| steps 0000100 | Validation Accuracy: 0.530
| steps 0000200 | loss: 0.212
| steps 0000200 | Validation Accuracy: 0.530
| steps 0000300 | loss: 0.136
| steps 0000300 | Validation Accuracy: 0.500
| steps 0000400 | loss: 0.135
| steps 0000400 | Validation Accuracy: 0.530
| steps 0000500 | loss: 0.216
| steps 0000500 | Validation Accuracy: 0.560
| steps 0000600 | loss: 0.118
| steps 0000600 | Validation Accuracy: 0.550
| steps 0000700 | loss: 0.228
| steps 0000700 | Validation Accuracy: 0.530
| steps 0000800 | loss: 0.195
| steps 0000800 | Validation Accuracy: 0.550
| steps 0000900 | loss: 0.266
| steps 0000900 | Validation Accuracy: 0.580
| steps 0001000 | loss: 0.232
| steps 0001000 | Validation Accuracy: 0.620
| steps 0001100 | loss: 0.153
| steps 0001100 | Validation Accuracy: 0.550
| steps 0001200 | loss: 0.216
| steps 0001200 | Validation Accuracy: 0.620
| steps 0001300 | loss: 0